<h1>Capstone project</h1>

Ok, the idea of the project is the following:

1. We take a list of all cities with more than 1 million inhabitants in north america (maybe include those in Europe and/or Australia) from Wikipedia https://en.wikipedia.org/wiki/List_of_North_American_cities_by_population (in case these are too many requests, we have to shrink down the number of cities)


2. Things become a bit tricky, we can in principle use the database of geonames at https://download.geonames.org/export/dump/ in order to get all the postal code areas of these places together with coordinates, but the names in wikipedia have to be translated into native languages (and "New York City" in Geonames is "New York").


3. We explore all these areas in Foursquare - the name conversion problem will appear again and I'm not sure yet how it can be fixed easily that the resulting areas do neither overlap - not miss too much of the cities. Well. The search for venues can be used without authentication, so it seems to be unlimited. If we get the shape of the cities, we can generate a hexagonal grid and get all the values. If there are issues with missing coordinates we can replace this by a triangular grid and triangulate all the venues.


4. The idea then is to generate a kind of fingerprint of each city by histograms and other graphs. E.g. the density of restaurants depending on the distance to the center. Of course, another problem arises here which is that some cities might have different number of "centers", but this might be given in the fingerprint. And sure, distances are related to a certain metric which is related to topology, infrastructure, no-go areas and so on. The different centers can be found using k-means and categorized to small, medium, large.


5. Maybe a higher-dimensional version of k-means can be helpful to compare and cluster all the cities


6. What are the differences between the cities? Is there a way to order them according to some dynamics (which shows their future classes with certain probabilities?)


7. Can we learn from those for their future evolution? (Of course, additional information on all levels would be quite helpful to get a better ordering, but this is only a small project ;-) e.g. fingerprints of areas could be correlated to crime rates and types)


8. Which areas need more restaurants, hotels, cafes, what so ever? This information is important for companies to know where to invest or for politicians to get an idea how the cities may develop due to certain decisions,...

<h1 align=center><font size = 5>Compare the largest western cities</font></h1>

## Introduction

...

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

As a first step, dependencies should be resolved.

In [50]:
#import numpy as np # library to handle data in a vectorized manner
#
import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#
#import json # library to handle JSON files
#
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#
from bs4 import BeautifulSoup
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#
## Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#
## import k-means from clustering stage
#from sklearn.cluster import KMeans
#
##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
#
print('Libraries imported.')

Libraries imported.


## Download and Explore Dataset

In [104]:
def getNorthAmerica():
    url = "https://en.wikipedia.org/wiki/List_of_North_American_cities_by_population"
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    # get relevant table
    table = soup.find(class_="wikitable")
    
    # get all rows
    rows = table.findChildren(['tr'])

    # get the headers
    cols=[]
    headers = rows[0].findChildren('th')
    for header in headers:
        cols.append(header.get_text(strip=True))
    
    cities=[]

    for row in rows:
        city = []
        cells = row.findChildren('td')
        for cell in cells:
            city.append(cell.get_text(strip=True))
            
        cities.append(city)

    df = pd.DataFrame(cities,columns=cols)
        
    return df

In [213]:
dfNA=getNorthAmerica()

In [214]:
dfNA.head()

,,City,Image,Country,Population,Year
0,None,None,None,None,None,None
1,1,Mexico City,,Mexico,"8,918,653[a][3][4]",2015[b]
2,2,New York City,,United States,"8,550,405[5]",2015
3,3,Los Angeles,,United States,"3,971,883[5]",2015
4,4,Toronto,,Canada,"2,826,498[c][7]",2015


In [215]:
dfNA.dropna(axis=0,inplace=True)
dfNA["Population"]=dfNA["Population"].apply(lambda x: x.split('[')[0])
dfNA["Year"]=dfNA["Year"].apply(lambda x: x.split('[')[0])
dfNA=dfNA.reset_index(drop=True)
dfNA["Population"]=dfNA["Population"].str.replace(",","").astype("int")

In [216]:
dfNA.head()

,,City,Image,Country,Population,Year
0,1,Mexico City,,Mexico,8918653,2015
1,2,New York City,,United States,8550405,2015
2,3,Los Angeles,,United States,3971883,2015
3,4,Toronto,,Canada,2826498,2015
4,5,Chicago,,United States,2720546,2015


In [219]:
dfNAmill=dfNA[dfNA["Population"]>1000000][["City","Country","Population"]].reset_index(drop=True)

In [220]:
dfNAmill

,City,Country,Population
0,Mexico City,Mexico,8918653
1,New York City,United States,8550405
2,Los Angeles,United States,3971883
3,Toronto,Canada,2826498
4,Chicago,United States,2720546
5,Houston,United States,2296224
6,Havana,Cuba,2117625
7,Montreal,Canada,1753034
8,Ecatepec de Morelos,Mexico,1677678
9,Philadelphia,United States,1567442


In [ ]:


https://nominatim.openstreetmap.org/search.php?q=Warsaw+Poland&polygon_geojson=1&format=json
    https://varunpant.com/posts/how-to-create-fishnets-or-geospatial-grids/